In [42]:
import requests
import bs4
import os
import re
import unicodedata
import pandas as pd
import nltk
import acquire as a
import utilities as util

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


import sys
sys.path.insert(1, '/Users/matthewdalton/')
from env import github_token, github_username

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords


In [24]:
url = ('https://github.com/microsoft?q=&type=&language=&sort=stargazers')
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html)

In [50]:
REPOS = []

In [26]:
for n in range(0,30):
    REPOS.append(soup.select('h3')[n].text)
    REPOS = list(set(REPOS))

In [49]:
35*30

1050

In [51]:
i = 35
while i < 80:
    url = 'https://github.com/microsoft?language=&page=' + str(i) + '&sort=stargazers&type='
    response = requests.get(url)
    html = response.text
    soup = bs4.BeautifulSoup(html)
    
    for n in range(0,30):
        REPOS.append(soup.select('h3')[n].text)
        REPOS = list(set(REPOS))
        
    i += 1

IndexError: list index out of range

In [53]:
len(REPOS)

1200

In [54]:
regexp = r'[\n]+\s+'
subject = REPOS[0]


re.sub(regexp, '', subject)

'demo-first-response-online\n'

In [55]:
repos = pd.Series(REPOS)

In [56]:
repos = repos.str.replace(regexp, "", regex=True)

In [57]:
repos = repos.str.replace('Archived\n', '').str.replace('\n', '')

In [58]:
REPOS = list('microsoft/' + repos)

In [59]:
REPOS

['microsoft/demo-first-response-online',
 'microsoft/sqlworkshops-azuresqlworkshop',
 'microsoft/coyote-samples',
 'microsoft/CNTK-docker',
 'microsoft/gatsby-starter-uifabric',
 'microsoft/LeadingConversationalSearchbySuggestingUsefulQuestions',
 'microsoft/python-program-analysis',
 'microsoft/calculator-specs',
 'microsoft/DWScripter',
 'microsoft/pmod',
 'microsoft/vscode-appcenter',
 'microsoft/bond-grpc-examples',
 'microsoft/xtreme-distil-transformers',
 'microsoft/AdventureWorksSkiApp',
 'microsoft/vsts-zendesk-app',
 'microsoft/VerifiableCredentials-Verification-SDK-Typescript',
 'microsoft/azure-repos-pr-multi-cherry-pick',
 'microsoft/nail_agent',
 'microsoft/OTP4LTE-U',
 'microsoft/dotnet-reliability',
 'microsoft/font-tools',
 'microsoft/MCW-SAP-NetWeaver-on-Azure',
 'microsoft/cortana-samples',
 'microsoft/KubeDevice',
 'microsoft/BotFramework-WeChat',
 'microsoft/CorrelationVector-Go',
 'microsoft/AzureSMR',
 'microsoft/azure_spatial_anchors_ros',
 'microsoft/logrl',
 'm

In [60]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

import sys
sys.path.insert(1, '/Users/matthewdalton/')

from env import github_token, github_username


headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

In [61]:
scrape_github_data()

Exception: Error response from github api! status code: 403, response: {"message": "API rate limit exceeded for user ID 48451646.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}

In [62]:
# f = open('data.json')
g = open('data2.json')

In [63]:
# returns JSON object as 
# a dictionary
# data = json.load(f)
data2 = json.load(g)

In [38]:
df = pd.DataFrame(data)

In [64]:
df2 = pd.DataFrame(data2)

In [39]:
df.head()

,repo,language,readme_contents
0,microsoft/vscode-java-test,Java,# Java Test Runner\n\n> Run and debug Java tes...
1,microsoft/checkedc-clang,C++,# The Checked C clang repo\n\nThis repo contai...
2,microsoft/azure-maven-plugins,Java,# Maven Plugins for Azure Services\n\nThis rep...
3,microsoft/vscode-node-debug,TypeScript,# Node Debug (legacy)\n\n[![build status](http...
4,microsoft/graphics-driver-samples,C++,# Overview\r\n\r\nThis project is focused on p...


In [67]:
df2.dropna(inplace=True)

In [69]:
df = pd.concat([df, df2], axis=0)

In [70]:
df.isnull().sum()

repo               0
language           0
readme_contents    0
dtype: int64

In [71]:
df.to_csv('microsoft_readmes.csv')

In [72]:
df.isnull().sum()

repo               0
language           0
readme_contents    0
dtype: int64

In [73]:
df.shape

(2031, 3)

In [79]:
df.language.value_counts(dropna=False)[0:15]

C#                  524
TypeScript          353
Python              234
C++                 202
JavaScript          156
PowerShell           99
Jupyter Notebook     73
C                    66
Java                 54
HTML                 47
Go                   31
Shell                24
CSS                  19
Objective-C          14
Dockerfile           13
Name: language, dtype: int64